<a href="https://colab.research.google.com/github/Hamoda-dabbit/Mining---classification-in-Arabic-Article/blob/main/CNN/xlnet_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# transformers - SOTA implementation of pretrained models
!pip install -U simpletransformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 249 kB 5.5 MB/s 
     |████████████████████████████████| 1.8 MB 35.4 MB/s 
     |████████████████████████████████| 365 kB 48.9 MB/s 
     |████████████████████████████████| 43 kB 1.6 MB/s 
     |████████████████████████████████| 4.7 MB 29.2 MB/s 
     |████████████████████████████████| 6.6 MB 46.5 MB/s 
     |████████████████████████████████| 1.2 MB 43.5 MB/s 
     |████████████████████████████████| 9.1 MB 23.2 MB/s 
     |████████████████████████████████| 596 kB 11.6 MB/s 
     |████████████████████████████████| 101 kB 5.9 MB/s 
     |████████████████████████████████| 181 kB 47.7 MB/s 
     |████████████████████████████████| 153 kB 47.6 MB/s 
     |████████████████████████████████| 63 kB 1.5 MB/s 
     |████████████████████████████████| 212 kB 46.4 MB/s 
     |████████████████████████████████| 140 kB 48.4 MB/s 
     |███████████████████████████████

In [2]:
# Importing libraries
import warnings
warnings.filterwarnings("ignore")
import nltk
nltk.download('stopwords')
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from nltk.stem.isri import ISRIStemmer
import string
from sklearn.utils import shuffle
import numpy as np
import re
import nltk
import pandas as pd
nltk.download('stopwords')
# Our dictionary will contain only of the top 7000 words appearing most frequently
top_words = 7000





# df=pd.read_excel('/content/dataset - Copy (2).xlsx')
df = pd.read_excel("drive/MyDrive/mining/1000.xlsx")

#df = df1.append(df2, ignore_index=True)
#df = df.append(df3, ignore_index=True)
#df = df.append(df4, ignore_index=True)
#df = df.append(df5, ignore_index=True)
#df = df.append(df6, ignore_index=True)

df = shuffle(df)
df

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,text,targe
2944,دعا البنك الدولي المغرب إلى اعتماد آليات لحماي...,2
21,شكلت استثناء في خارطة الإعلام العربي لعدم اعتر...,0
1338,أيدت الغرفة الجنحية الاستئنافية بالمحكمة الابت...,1
1483,لم يؤاخذ قسم الجرائم المالية الابتدائي باستئنا...,1
1645,عدد المعتقلين في الشبكة بلغ وترقب تحقيق من الق...,1
...,...,...
3674,أطلق الشيخ حماد القباج تصريحات قوية من شأنها أ...,3
3106,أكدت نبيلة منيب وكيلة اللائحة الوطنية لفيدرالي...,3
3635,يعيش مجلس المستشارين على وقع خلافات حادة حول ت...,3
1224,أعاد أربعة شباب تمثيل جريمة هزت منطقة حامة »، ...,1


In [3]:
print(np.sum(df.isnull().any(axis=1)))

clean_df= df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))
#-------------
print (df.notnull().any(axis = 0))
print(len(df))
#-------------
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
      df[col] = df[col].astype(str).str.replace(letter,'', regex=True)
        
remove_hashtag(clean_df)
clean_df
#-------------
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)
#-------------
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text
#-------------    
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)
#-------------
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
print(clean_df)

12
0
text     True
targe    True
dtype: bool
5000
                                                   text  targe
2944  دعا البنك الدولي المغرب الي اعتماد اليات لحماي...      2
21    شكلت استثناء في خارطه الاعلام العربي لعدم اعتر...      0
1338  ايدت الغرفه الجنحيه الاستئنافيه بالمحكمه الابت...      1
1483  لم يؤاخذ قسم الجرائم الماليه الابتدائي باستئنا...      1
1645  عد المعتقلين في الشبكه بلغ وترقب تحقيق من القي...      1
...                                                 ...    ...
3674  اطلق الشيخ حماد القباج تصريحات قويه من شانها ا...      3
3106  اكدت نبيله منيب وكيله الائحه الوطنيه لفيدراليه...      3
3635  يعيش مجلس المستشارين علي وقع خلافات حاده حول ت...      3
1224  اعاد اربعه شباب تمثيل جريمه هزت منطقه حامه » ب...      1
1542  وجهت له طعنه ب منجل اصابته في العنقيستمع قاضي ...      1

[4988 rows x 2 columns]


In [4]:
clean_df.targe.value_counts()

0    1000
1    1000
4    1000
3    1000
2     988
Name: targe, dtype: int64

In [ ]:

# #df=df.dropna()
# # X,y= clean_df['text'],clean_df['targe'] 


# y = clean_df['targe']
# X = clean_df['text']
# stopwords_list = stopwords.words('arabic')

# # tfidf = TfidfVectorizer(max_features=1500, min_df=5, max_df=0.7,token_pattern=r"(?u)\b\w\w+\b",stop_words=stopwords_list)
# #documents = [word for word in documents if not word in stopwords.words()]
# from sklearn.feature_extraction.text import CountVectorizer           #TF  الطريقة التي تقوم بالعد
# vectorizer = CountVectorizer(max_features=10000,min_df=5,max_df=0.7,stop_words=stopwords_list)
# X = vectorizer.fit_transform(X).toarray()
# #print(X)


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  % sorted(inconsistent)


In [5]:
# from tensorflow.keras.preprocessing.text import Tokenizer
# X = clean_df['text']
# x1 = clean_df['text'].values
# t = Tokenizer(oov_token='<UNK>')
# # fit the tokenizer on the documents
# t.fit_on_texts(x1)
# t.word_index['<PAD>'] = 0

# VOCAB_SIZE = len(t.word_index)


In [ ]:
# from tensorflow.keras.preprocessing import sequence
# MAX_SEQUENCE_LENGTH = 1000
# train_sequences = t.texts_to_sequences(X)
# # test_sequences = t.texts_to_sequences(norm_test_reviews)
# X_train = sequence.pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
# # X_test = sequence.pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [6]:
# from nltk.tokenize import RegexpTokenizer
# tokenizer = RegexpTokenizer(r'\w+')
# clean_df["text"] = clean_df["text"].apply(tokenizer.tokenize)
# # print(clean_df['text'].values[0])
# clean_df

,text,targe
2944,"[دعا, البنك, الدولي, المغرب, الي, اعتماد, اليا...",2
21,"[شكلت, استثناء, في, خارطه, الاعلام, العربي, لع...",0
1338,"[ايدت, الغرفه, الجنحيه, الاستئنافيه, بالمحكمه,...",1
1483,"[لم, يؤاخذ, قسم, الجرائم, الماليه, الابتدائي, ...",1
1645,"[عد, المعتقلين, في, الشبكه, بلغ, وترقب, تحقيق,...",1
...,...,...
3674,"[اطلق, الشيخ, حماد, القباج, تصريحات, قويه, من,...",3
3106,"[اكدت, نبيله, منيب, وكيله, الائحه, الوطنيه, لف...",3
3635,"[يعيش, مجلس, المستشارين, علي, وقع, خلافات, حاد...",3
1224,"[اعاد, اربعه, شباب, تمثيل, جريمه, هزت, منطقه, ...",1


In [12]:
# Total dataset length

X = clean_df["text"]
y = clean_df['targe']


dataset_length = df.shape[0]

# Training dataset length
split = int(dataset_length * 0.25)
split
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

train_df = pd.DataFrame(X_train)
train_df['target'] = y_train

eval_df = pd.DataFrame(X_test)
eval_df['target'] = y_test

In [ ]:
eval_df


In [14]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import logging
import sklearn


logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

# They are lot of arguments to play with
'''
args = {
   'output_dir': 'outputs/',
   'cache_dir': 'cache/',
   'fp16': True,
   'fp16_opt_level': 'O1',
   'max_seq_length': 256,
   'train_batch_size': 8,
   'eval_batch_size': 8,
   'gradient_accumulation_steps': 1,
   'num_train_epochs': 3,
   'weight_decay': 0,
   'learning_rate': 4e-5,
   'adam_epsilon': 1e-8,
   'warmup_ratio': 0.06,
   'warmup_steps': 0,
   'max_grad_norm': 1.0,
   'logging_steps': 50,
   'evaluate_during_training': False,
   'save_steps': 2000,
   'eval_all_checkpoints': True,
   'use_tensorboard': True,
   'overwrite_output_dir': True,
   'reprocess_input_data': False,
}
'''

# Create a ClassificationModel
model = ClassificationModel('xlnet', 'xlnet-base-cased',num_labels=5, args={'num_train_epochs':1, 'train_batch_size':32, 'max_seq_length':128},use_cuda=False) # You can set class weights by using the optional weight argument

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.bias', 'lm_loss.weight']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.bias', 'sequence_summary.summary.weight', 'logits_proj.bias', 'logits_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

In [ ]:
train_df

In [16]:

# Train the model
model.train_model(train_df)

  0%|          | 0/1250 [00:00<?, ?it/s]

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/40 [00:00<?, ?it/s]

(40, 1.624020367860794)

In [17]:

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df, acc=sklearn.metrics.accuracy_score)

  0%|          | 0/3738 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/468 [00:00<?, ?it/s]

In [18]:
result

{'acc': 0.20224719101123595,
 'eval_loss': 1.6127002715555012,
 'mcc': 7.614484951944167e-05}

In [19]:
print(result)

{'mcc': 7.614484951944167e-05, 'acc': 0.20224719101123595, 'eval_loss': 1.6127002715555012}
